In [3]:
import yaml
import sqlite3
from monad_std import Option, std_types

In [2]:
fp = r'E:\WBH\Games\eve forever\EVE Data\sde\sde\fsd\typeIDs.yaml'
with open(fp, 'r', encoding='utf-8') as f:
    type_ids = yaml.load(f, yaml.CLoader)

In [8]:
id_and_name = {}
for type_id, type_data in type_ids.items():
    name = None
    if 'name' in type_data.keys():
        if 'zh' in type_data['name'].keys():
            name = type_data['name']['zh']
        elif 'en' in type_data['name'].keys():
            name = type_data['name']['en']

    if not name:
        name = f"无名称物品（{type_id}）"
    name = name.replace(' (Do not translate)', '').replace('DO NOT TRANSLATE - ', '')
    id_and_name[type_id] = (name,
                            std_types.MDict(**type_data)
                            .get("published")
                            .map(lambda x: 1 if x else 0)
                            .unwrap_unchecked())

In [5]:
db = sqlite3.connect('id-name.db')
cursor = db.cursor()

In [6]:
_sql = """create table if not exists typename (
    type_id integer primary key not null,
    type_name text not null,
    published integer not null default 1
);"""
cursor.execute(_sql)
db.commit()

In [9]:
_sql = "insert into typename values (?, ?, ?);"
for key, value in id_and_name.items():
    cursor.execute(_sql, (key, value[0], value[1]))
db.commit()

In [30]:
db.close()